# Some Simple PagerDuty Data Pulls

## Get API keys from `.env` and declare headers for re-use

In [1]:
import httpx
import polars as pl
import dotenv

# Note: a `.env` file needs to be made in the root.  The `data/template.env` serves as a template and the `just init` command will auto-copy it, but requires the personal key be specified, of course.
personal_api_key = dotenv.get_key('../.env', 'PAGERDUTY_REST_API_KEY')
public_demonstration_api_key = dotenv.get_key('../.env', 'PAGERDUTY_API_CONCEPTS_EXAMPLE_KEY')


token_request_prefix = "Token token="

# try headers for personal repo
if personal_api_key is not None:
    headers_personal = {
        "Accept": "application/json",
        "Authorization": (token_request_prefix + personal_api_key),
        "Content-Type": "application/json"
    }
else:
    print("PAGERDUTY_REST_API_KEY not found.  Check whether declared in `.env` and dotenv pathing")

# try headers for public example repo
if public_demonstration_api_key is not None:
    headers_publicizable_example = {
        "Accept": "application/json",
        "Authorization": (token_request_prefix + public_demonstration_api_key),
        "Content-Type": "application/json"
    }
else:
    print("PAGERDUTY_API_CONCEPTS_EXAMPLE_KEY not found.  Check whether `.env` was copied from `data/template.env` and dotenv pathing")


# declare default `headers` value
if headers_personal is not None:
    headers = headers_personal
    print("`headers` set to `headers_personal` :)")
elif headers_publicizable_example is not None:
    headers = headers_publicizable_example
    print("`headers` set to `headers_publicizable_example` :)")
else:
    print("No `headers` variable set.  Code below will not run.")

`headers` set to `headers_personal` :)


In [2]:
# Setting Base URL
base_url = "https://api.pagerduty.com"
base_url

'https://api.pagerduty.com'

#### Manually Switch Headers to Example here if you like

In [3]:
#headers = headers_personal
headers = headers_publicizable_example

## API Calls to DataFrames

### GET Users

In [4]:
endpoint = "users"
url  = base_url + "/" + endpoint

with httpx.Client() as client:
    getuser_response = client.get(url, headers=headers)

print(f"GET ../users: {getuser_response}\n")

if getuser_response.is_success:
 
    dict = getuser_response.json()
    # print(dict)
    users_df = pl.DataFrame(dict.get(endpoint,[]))
    print(f"Was success!: {users_df.head(5)}\n")
else:
    print("Was NOT a success.")
    print(f"Error: {getuser_response.status_code}")

GET ../users: <Response [200 OK]>

Was success!: shape: (5, 19)
┌────────────┬────────────┬────────────┬────────────┬───┬──────┬───────────┬───────────┬───────────┐
│ name       ┆ email      ┆ time_zone  ┆ color      ┆ … ┆ type ┆ summary   ┆ self      ┆ html_url  │
│ ---        ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---  ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆ str        ┆ str        ┆ str        ┆   ┆ str  ┆ str       ┆ str       ┆ str       │
╞════════════╪════════════╪════════════╪════════════╪═══╪══════╪═══════════╪═══════════╪═══════════╡
│ Alan B.    ┆ alan.shepa ┆ America/Lo ┆ blue-viole ┆ … ┆ user ┆ Alan B.   ┆ https://a ┆ https://p │
│ Shepard,   ┆ rd@nasa.ex ┆ s_Angeles  ┆ t          ┆   ┆      ┆ Shepard,  ┆ pi.pagerd ┆ dt-apidoc │
│ Jr.        ┆ ample      ┆            ┆            ┆   ┆      ┆ Jr.       ┆ uty.com/u ┆ s.pagerdu │
│            ┆            ┆            ┆            ┆   ┆      ┆           ┆ sers/…    ┆ ty.co…    │
│ Aleksei    ┆ aleksei.so ┆

### GET Licenses

In [5]:
endpoint = "licenses"
url  = base_url + "/" + endpoint
with httpx.Client() as client:
    response = client.get(url, headers=headers)

print(f"GET ../licenses response: {response}\n")

if response.is_success:
    
    dict = response.json()
    #print(dict)
    licenses_df = pl.DataFrame(dict.get(endpoint,[]))
    print(f"Was success!: {licenses_df}\n")
else:
    print("Was NOT a success.")
    print(f"Error: {getuser_response.status_code}")

GET ../licenses response: <Response [200 OK]>

Was success!: shape: (2, 11)
┌─────────────┬─────────────┬────────────┬──────────┬───┬──────┬────────────┬─────────┬────────────┐
│ allocations ┆ current_val ┆ descriptio ┆ html_url ┆ … ┆ self ┆ summary    ┆ type    ┆ valid_role │
│ _available  ┆ ue          ┆ n          ┆ ---      ┆   ┆ ---  ┆ ---        ┆ ---     ┆ s          │
│ ---         ┆ ---         ┆ ---        ┆ null     ┆   ┆ null ┆ str        ┆ str     ┆ ---        │
│ i64         ┆ i64         ┆ str        ┆          ┆   ┆      ┆            ┆         ┆ list[str]  │
╞═════════════╪═════════════╪════════════╪══════════╪═══╪══════╪════════════╪═════════╪════════════╡
│ 106         ┆ 44          ┆            ┆ null     ┆ … ┆ null ┆ Digital    ┆ license ┆ ["owner",  │
│             ┆             ┆            ┆          ┆   ┆      ┆ Operations ┆         ┆ "admin", … │
│             ┆             ┆            ┆          ┆   ┆      ┆ (Full      ┆         ┆ "restricte │
│             ┆

### GET License_Allocations

In [29]:
endpoint = "license_allocations"
url  = base_url + "/" + endpoint

with httpx.Client() as client:
    response = client.get(url, headers=headers)
    
print(f"GET ..//license_allocations: {response}\n")

if response.is_success:
    dict = response.json()
    lic_allocs = dict.get(endpoint,[])
    lic_allocs_df = pl.DataFrame(lic_allocs)
    print(f"Was success!: {lic_allocs_df}\n")
else:
    print("Was NOT a success.")
    print(f"Error: {getuser_response.status_code}")

GET ..//license_allocations: <Response [200 OK]>

Was success!: shape: (48, 3)
┌─────────────────────┬───────────────────────────────────┬──────────────────────────────┐
│ allocated_at        ┆ license                           ┆ user                         │
│ ---                 ┆ ---                               ┆ ---                          │
│ str                 ┆ struct[9]                         ┆ struct[2]                    │
╞═════════════════════╪═══════════════════════════════════╪══════════════════════════════╡
│ 2022-01-20T10:07:36 ┆ {"",null,"PHAWL6E","Digital Oper… ┆ {"PPC00ZX","user_reference"} │
│ 2022-01-20T10:07:37 ┆ {"",null,"PHAWL6E","Digital Oper… ┆ {"PCA13N7","user_reference"} │
│ 2022-01-20T10:07:37 ┆ {"",null,"PHAWL6E","Digital Oper… ┆ {"PVRS1IU","user_reference"} │
│ 2022-01-20T10:07:37 ┆ {"",null,"PHAWL6E","Digital Oper… ┆ {"PRF4164","user_reference"} │
│ …                   ┆ …                                 ┆ …                            │
│ 2022-01-2

## Data Display & Exploration

In [14]:
users_df.sample(1)

name,email,time_zone,color,avatar_url,billed,role,description,invitation_sent,job_title,teams,contact_methods,notification_rules,coordinated_incidents,id,type,summary,self,html_url
str,str,str,str,str,bool,str,null,bool,null,list[struct[5]],list[struct[5]],list[struct[5]],list[null],str,str,str,str,str
"""Andriyan Nikol…","""andriyan.nikol…","""America/Los_An…","""chocolate""","""https://secure…",true,"""limited_user""",null,true,null,"[{""PGVXG6U"",""team_reference"",""Space Cosmonauts"",""https://api.pagerduty.com/teams/PGVXG6U"",""https://pdt-apidocs.pagerduty.com/teams/PGVXG6U""}]","[{""PBYXCWL"",""email_contact_method_reference"",""Default"",""https://api.pagerduty.com/users/PIQHXK2/contact_methods/PBYXCWL"",null}, {""PAIL03R"",""phone_contact_method_reference"",""Work"",""https://api.pagerduty.com/users/PIQHXK2/contact_methods/PAIL03R"",null}, … {""PTAOLBB"",""sms_contact_method_reference"",""Mobile"",""https://api.pagerduty.com/users/PIQHXK2/contact_methods/PTAOLBB"",null}]","[{""PY24XV5"",""assignment_notification_rule_reference"",""0 minutes: channel PBYXCWL"",""https://api.pagerduty.com/users/PIQHXK2/notification_rules/PY24XV5"",null}, {""PBBSY7V"",""assignment_notification_rule_reference"",""0 minutes: channel PBYXCWL"",""https://api.pagerduty.com/users/PIQHXK2/notification_rules/PBBSY7V"",null}, … {""PJ6PIGN"",""assignment_notification_rule_reference"",""0 minutes: channel PTAOLBB"",""https://api.pagerduty.com/users/PIQHXK2/notification_rules/PJ6PIGN"",null}]",[],"""PIQHXK2""","""user""","""Andriyan Nikol…","""https://api.pa…","""https://pdt-ap…"


In [ ]:
licenses_df.head(2)

In [11]:
lic_allocs_df.tail(3)

allocated_at,license,user
str,struct[9],struct[2]
"""2022-01-20T10:…","{"""",null,""PE18KUR"",""Digital Operations (Stakeholder)"",""Stakeholder"",null,""Digital Operations (Stakeholder)"",""license"",[""read_only_user"", ""read_only_limited_user""]}","{""P7WUMER"",""user_reference""}"
"""2022-01-20T10:…","{"""",null,""PE18KUR"",""Digital Operations (Stakeholder)"",""Stakeholder"",null,""Digital Operations (Stakeholder)"",""license"",[""read_only_user"", ""read_only_limited_user""]}","{""P3K6M21"",""user_reference""}"
"""2022-01-20T10:…","{"""",null,""PE18KUR"",""Digital Operations (Stakeholder)"",""Stakeholder"",null,""Digital Operations (Stakeholder)"",""license"",[""read_only_user"", ""read_only_limited_user""]}","{""PDEHYI9"",""user_reference""}"


In [12]:
print(lic_allocs_df.select("license"))

shape: (48, 1)
┌───────────────────────────────────┐
│ license                           │
│ ---                               │
│ struct[9]                         │
╞═══════════════════════════════════╡
│ {"",null,"PHAWL6E","Digital Oper… │
│ {"",null,"PHAWL6E","Digital Oper… │
│ {"",null,"PHAWL6E","Digital Oper… │
│ {"",null,"PHAWL6E","Digital Oper… │
│ …                                 │
│ {"",null,"PE18KUR","Digital Oper… │
│ {"",null,"PE18KUR","Digital Oper… │
│ {"",null,"PE18KUR","Digital Oper… │
│ {"",null,"PE18KUR","Digital Oper… │
└───────────────────────────────────┘


In [45]:
print(lic_allocs_df.select("user"))

shape: (48, 1)
┌──────────────────────────────┐
│ user                         │
│ ---                          │
│ struct[2]                    │
╞══════════════════════════════╡
│ {"PPC00ZX","user_reference"} │
│ {"PCA13N7","user_reference"} │
│ {"PVRS1IU","user_reference"} │
│ {"PRF4164","user_reference"} │
│ …                            │
│ {"PZ0D2XH","user_reference"} │
│ {"P7WUMER","user_reference"} │
│ {"P3K6M21","user_reference"} │
│ {"PDEHYI9","user_reference"} │
└──────────────────────────────┘


### Explode, Implode, Flatten, other...?

In [66]:
lic_allocs_df.select(pl.col("user")).to_dict()

{'user': shape: (48,)
 Series: 'user' [struct[2]]
 [
 	{"PPC00ZX","user_reference"}
 	{"PCA13N7","user_reference"}
 	{"PVRS1IU","user_reference"}
 	{"PRF4164","user_reference"}
 	{"PO5G0TE","user_reference"}
 	{"PRJ4208","user_reference"}
 	{"PVTOBQ4","user_reference"}
 	{"PRH0AAE","user_reference"}
 	{"PO7CZLR","user_reference"}
 	{"P0PFXHO","user_reference"}
 	{"P247ZMF","user_reference"}
 	{"PL0TA9M","user_reference"}
 	…
 	{"PWHSPDK","user_reference"}
 	{"PT84P1X","user_reference"}
 	{"PPYGOP7","user_reference"}
 	{"P8MVNZH","user_reference"}
 	{"PJ29Q1C","user_reference"}
 	{"PFSLPPP","user_reference"}
 	{"PBGAOZM","user_reference"}
 	{"PY7PNKZ","user_reference"}
 	{"PPF7HYE","user_reference"}
 	{"PZ0D2XH","user_reference"}
 	{"P7WUMER","user_reference"}
 	{"P3K6M21","user_reference"}
 	{"PDEHYI9","user_reference"}
 ]}

## ... What comes Next? :)